In [5]:
def recommandation(tconst):
    import pandas as pd
    from sklearn.neighbors import NearestNeighbors

    import warnings
    warnings.filterwarnings("ignore", category=UserWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)

    df_test = pd.read_csv("../machine learning/DF_ML.csv.gz")

    # ----------------------------------------------------------
    # Préparation des données
    # ----------------------------------------------------------

    index = df_test.index
    df_test_num = df_test.select_dtypes('number')
    df_test_cat = df_test.select_dtypes(['object', 'category', 'string', 'bool'])

    # Normalisation des colonnes numériques
    from sklearn.preprocessing import MinMaxScaler
    SN = MinMaxScaler()
    df_test_num_SN = pd.DataFrame(SN.fit_transform(df_test_num), columns=df_test_num.columns, index=index)

    # Encodage uniquement de la colonne 'nconst'
    df_test_cat_encoded = df_test_cat.copy()
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    df_test_cat_encoded['nconst'] = le.fit_transform(df_test_cat_encoded['nconst'].fillna('inconnu'))

    # On assemble le df numérique et le df texte
    df_test_encoded = pd.concat([df_test_num_SN, df_test_cat_encoded], axis=1)

    #On sépare notre df en deux groupes, en fonction de la note
    bons_films = df_test_encoded[df_test_encoded['notes'] >= 0.7]

    # ----------------------------------------------------------
    # KNN sur les caractéristiques numériques
    # ----------------------------------------------------------

    colonnes_a_exclure = ['tconst', 'title', 'tmdb_popularity', 'title_ratings_numVotes', 'imdb_id', 'genres', 'overview', 'overview_lem']
    caracteristiques = df_test_encoded.columns.drop(colonnes_a_exclure, errors='ignore')

    model = NearestNeighbors(n_neighbors=1000, metric='euclidean') # Il faudra tester d'autres combinaisons
    model.fit(bons_films[caracteristiques])

    caract_film = df_test_encoded[df_test_encoded['tconst'] == tconst][caracteristiques]
    distances, indices = model.kneighbors(caract_film)

    if not df_test_encoded[(df_test_encoded['tconst'] == tconst) & (df_test_encoded['notes'] >= 0.7)].empty:
        selection = bons_films.iloc[indices[0]].copy()
        selection['distance_knn'] = distances[0]
    else:
        selection = bons_films.iloc[indices[0]].copy()
        selection = pd.concat([df_test_encoded[df_test_encoded['tconst'] == tconst], selection.iloc[:-1]], axis=0)
        selection['distance_knn'] = distances[0]

    # ----------------------------------------------------------
    # TF-IDF avec lemmatisation
    # ----------------------------------------------------------

    colonnes_poids = {
        'genres': 1,
        'overview_lem': 1,
        'nconst': 1
    }

    # On crée une colonne 'texte' qui combine les valeurs pondérées des colonnes spécifiées
    selection['texte'] = selection.apply(lambda row: 
        ' '.join([
            (str(row[col]) + ' ') * colonnes_poids.get(col, 1)  # Répète la valeur de la colonne selon son poids
            for col in colonnes_poids.keys()
        ]),
        axis=1
    )

    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8)
    tfidf_matrix = vectorizer.fit_transform(selection['texte'])

    model_tfidf = NearestNeighbors(n_neighbors=1000, metric='cosine')
    model_tfidf.fit(tfidf_matrix)

    distances_tfidf, indices_tfidf = model_tfidf.kneighbors(tfidf_matrix[0])
    selection['distance_tfidf'] = distances_tfidf[0]

    # ----------------------------------------------------------
    # Moyenne pondérée des distances
    # ----------------------------------------------------------

    poids_knn = 1
    poids_tfidf = 100

    selection['distance_ponderee'] = (
        poids_knn * selection['distance_knn']) + (poids_tfidf * selection['distance_tfidf']
    ) 

    # Tri final par la distance pondérée
    selection = selection.sort_values(by='distance_ponderee')

    # ----------------------------------------------------------
    # Résultat final
    # ----------------------------------------------------------
    selection_finale = pd.DataFrame(selection['tconst'][1:11]).reset_index(drop=True)

    return selection_finale

In [6]:
recommandation('tt0099785')

,tconst
0,tt0107614
1,tt0241527
2,tt0295297
3,tt0243133
4,tt10095582
5,tt0335266
6,tt0159097
7,tt0077416
8,tt0083652
9,tt0115005
